In [66]:
import os

In [71]:
import os
import json
import pandas as pd
import traceback
from langchain.chat_models import ChatOpenAI
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback
import PyPDF2
from dotenv import load_dotenv

In [72]:
load_dotenv()

True

In [73]:
key = os.getenv("OPENAI_API_KEY")

In [97]:
t = os.getenv("OPENAI_API_KEY2")

In [98]:
print(t)

None


In [75]:
key = ''

In [76]:
llm = ChatOpenAI(openai_api_key = key, model_name = 'gpt-3.5-turbo', temperature = 0.7)

In [77]:
llm

ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x00000215BA2D9290>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x00000215BA1A5F50>, openai_api_key='sk-EJdndl6NxfpBUOBMa9IkzNDlQtx8OmMKBe19gzovFTT3BlbkFJUVpfC-PvySJdbIvkF0nADYBTz4aAv6Mh1X_T6rIccA', openai_proxy='')

In [78]:
with open("C:/Users/hp/OneDrive/Desktop/5thS/iNeuron Lab/GenAI-Langchain/Response.json","r") as f:
    RESPONSE_JSON = json.load(f)

In [79]:
RESPONSE_JSON

{'1': {'mcq': 'multiple choice question',
  'options': {'a': 'choice here',
   'b': 'choice here',
   'c': 'choice here',
   'd': 'choice here'},
  'correct': 'correct answer'},
 '2': {'mcq': 'multiple choice question',
  'options': {'a': 'choice here',
   'b': 'choice here',
   'c': 'choice here',
   'd': 'choice here'},
  'correct': 'correct answer'},
 '3': {'mcq': 'multiple choice question',
  'options': {'a': 'choice here',
   'b': 'choice here',
   'c': 'choice here',
   'd': 'choice here'},
  'correct': 'correct answer'}}

In [80]:
TEMPLATE="""
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz  of {number} multiple choice questions for {subject} students in {tone} tone. 
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like  RESPONSE_JSON below  and use it as a guide. \
Ensure to make {number} MCQs
### RESPONSE_JSON
{RESPONSE_JSON}

"""

In [81]:
quiz_generation_prompt = PromptTemplate(
    input_variables=['text','numbers','subject','tone','RESPONSE_JSON'],
    template=TEMPLATE
)

In [82]:
quiz_chain = LLMChain(llm=llm,prompt = quiz_generation_prompt, output_key = "quiz", verbose = True )

In [83]:
TEMPLATE2="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [84]:
quiz_evaluation_prompt=PromptTemplate(input_variables=["subject", "quiz"], template=TEMPLATE2)


In [85]:
review_chain=LLMChain(llm=llm, prompt=quiz_evaluation_prompt, output_key="review", verbose=True)


In [86]:
generate_evaluate_chain=SequentialChain(chains=[quiz_chain, review_chain], input_variables=["text", "number", "subject", "tone", "RESPONSE_JSON"],
                                        output_variables=["quiz", "review"], verbose=True,)


In [87]:
with open("C:/Users/hp/OneDrive/Desktop/5thS/iNeuron Lab/GenAI-Langchain/data.txt",'r') as f:
    TEXT = f.read()

In [88]:
generate_evaluate_chain

SequentialChain(verbose=True, chains=[LLMChain(verbose=True, prompt=PromptTemplate(input_variables=['RESPONSE_JSON', 'number', 'subject', 'text', 'tone'], template='\nText:{text}\nYou are an expert MCQ maker. Given the above text, it is your job to create a quiz  of {number} multiple choice questions for {subject} students in {tone} tone. \nMake sure the questions are not repeated and check all the questions to be conforming the text as well.\nMake sure to format your response like  RESPONSE_JSON below  and use it as a guide. Ensure to make {number} MCQs\n### RESPONSE_JSON\n{RESPONSE_JSON}\n\n'), llm=ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x00000215BA2D9290>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x00000215BA1A5F50>, openai_api_key='sk-EJdndl6NxfpBUOBMa9IkzNDlQtx8OmMKBe19gzovFTT3BlbkFJUVpfC-PvySJdbIvkF0nADYBTz4aAv6Mh1X_T6rIccA', openai_proxy=''), output_key='quiz'), LLMChain(verbose=True, prompt=PromptTemplate(inp

In [89]:
import json
json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [104]:
with get_openai_callback() as cb:
    response = generate_evaluate_chain(
    {
        'text' : TEXT,
        'number' : 5,
        'subject': 'AI',
        'tone' : 'Simple',
        'RESPONSE_JSON': json.dumps(RESPONSE_JSON)
        
    }
    )



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text:Generative artificial intelligence (generative AI, GenAI,[1] or GAI) is artificial intelligence capable of generating text, images, videos, or other data using generative models,[2] often in response to prompts.[3][4] Generative AI models learn the patterns and structure of their input training data and then generate new data that has similar characteristics.[5][6]

Improvements in transformer-based deep neural networks, particularly large language models (LLMs), enabled an AI boom of generative AI systems in the early 2020s. These include chatbots such as ChatGPT, Copilot, Gemini and LLaMA, text-to-image artificial intelligence image generation systems such as Stable Diffusion, Midjourney and DALL-E, and text-to-video AI generators such as Sora.[7][8][9][10] Companies such as OpenAI, Anthropic, Microsoft, Google, and Baidu as well as numerous smaller firms have developed generat

In [99]:
print(f"Total Tokens:{cb.total_tokens}")
print(f"Prompt Tokens:{cb.prompt_tokens}")
print(f"Completion Tokens:{cb.completion_tokens}")
print(f"Total Cost:{cb.total_cost}")

Total Tokens:1751
Prompt Tokens:1385
Completion Tokens:366
Total Cost:0.0028095


In [102]:
cb

Tokens Used: 1751
	Prompt Tokens: 1385
	Completion Tokens: 366
Successful Requests: 2
Total Cost (USD): $0.0028095

In [103]:
RESPONSE_JSON

{'1': {'mcq': 'multiple choice question',
  'options': {'a': 'choice here',
   'b': 'choice here',
   'c': 'choice here',
   'd': 'choice here'},
  'correct': 'correct answer'},
 '2': {'mcq': 'multiple choice question',
  'options': {'a': 'choice here',
   'b': 'choice here',
   'c': 'choice here',
   'd': 'choice here'},
  'correct': 'correct answer'},
 '3': {'mcq': 'multiple choice question',
  'options': {'a': 'choice here',
   'b': 'choice here',
   'c': 'choice here',
   'd': 'choice here'},
  'correct': 'correct answer'}}

In [106]:
quiz = response.get("quiz")

In [108]:
quiz = json.loads(quiz)

In [109]:
quiz

{'1': {'mcq': 'When was the academic discipline of artificial intelligence established?',
  'options': {'a': '1950', 'b': '1956', 'c': '1960', 'd': '1970'},
  'correct': 'b'},
 '2': {'mcq': 'What type of models do generative AI systems use to generate new data?',
  'options': {'a': 'Supervised learning models',
   'b': 'Unsupervised learning models',
   'c': 'Generative models',
   'd': 'Reinforcement learning models'},
  'correct': 'c'},
 '3': {'mcq': 'Which company developed the chatbot ChatGPT?',
  'options': {'a': 'OpenAI', 'b': 'Microsoft', 'c': 'Google', 'd': 'Baidu'},
  'correct': 'a'},
 '4': {'mcq': 'What ethical concern has been raised regarding the use of generative AI?',
  'options': {'a': 'Increasing human creativity',
   'b': 'Potential misuse for cybercrime',
   'c': 'Enhancing customer service',
   'd': 'Improving healthcare'},
  'correct': 'b'},
 '5': {'mcq': 'Who created the computer program AARON to generate paintings in the early 1970s?',
  'options': {'a': 'Harold C

In [110]:
quiz_table_data = []
for key, value in quiz.items():
    mcq = value["mcq"]
    options = " | ".join(
        [
            f"{option}: {option_value}"
            for option, option_value in value["options"].items()
            ]
        )
    correct = value["correct"]
    quiz_table_data.append({"MCQ": mcq, "Choices": options, "Correct": correct})

In [111]:
quiz_table_data

[{'MCQ': 'When was the academic discipline of artificial intelligence established?',
  'Choices': 'a: 1950 | b: 1956 | c: 1960 | d: 1970',
  'Correct': 'b'},
 {'MCQ': 'What type of models do generative AI systems use to generate new data?',
  'Choices': 'a: Supervised learning models | b: Unsupervised learning models | c: Generative models | d: Reinforcement learning models',
  'Correct': 'c'},
 {'MCQ': 'Which company developed the chatbot ChatGPT?',
  'Choices': 'a: OpenAI | b: Microsoft | c: Google | d: Baidu',
  'Correct': 'a'},
 {'MCQ': 'What ethical concern has been raised regarding the use of generative AI?',
  'Choices': 'a: Increasing human creativity | b: Potential misuse for cybercrime | c: Enhancing customer service | d: Improving healthcare',
  'Correct': 'b'},
 {'MCQ': 'Who created the computer program AARON to generate paintings in the early 1970s?',
  'Choices': 'a: Harold Cohen | b: Alan Turing | c: Daedalus | d: Hero of Alexandria',
  'Correct': 'a'}]

In [112]:
quiz = pd.DataFrame(quiz_table_data)

In [113]:
quiz

,MCQ,Choices,Correct
0,When was the academic discipline of artificial...,a: 1950 | b: 1956 | c: 1960 | d: 1970,b
1,What type of models do generative AI systems u...,a: Supervised learning models | b: Unsupervise...,c
2,Which company developed the chatbot ChatGPT?,a: OpenAI | b: Microsoft | c: Google | d: Baidu,a
3,What ethical concern has been raised regarding...,a: Increasing human creativity | b: Potential ...,b
4,Who created the computer program AARON to gene...,a: Harold Cohen | b: Alan Turing | c: Daedalus...,a


In [115]:
quiz.to_csv("GENAI-MCQS.csv")